# BDENG project SS2024

## Topic

The main goal is to check if there is correlation between CO2 emissions and between the sum of new registered/used cars in Austria (since 2000).

For BDENG project we want to get car data from various sources and check if they correspond with the available data between various source. We got one new and used car registration set and we want to check if the count corresponds with the publicly available statistics from https://www.statistik.at/statistiken/tourismus-und-verkehr/fahrzeuge/kfz-bestand which got webscrape.
Since we didnt found an good fitting API, we wrote our own.

Members: Grégoire Bartek, Matthias Huber, David Berger


# Data Sources

## Webscraping

Car count of registrations (Webscraping)

https://www.statistik.at/statistiken/tourismus-und-verkehr/fahrzeuge/kfz-bestand

## CSV

emission data: https://www.data.gv.at/katalog/dataset/bd462a04-2453-4c76-95bb-cfbd043540f5#resources . For the emission data it was important to use emission data only from passenger vehicles, since we get the registration data from normal "PKWs". The NFR emission category for passenger vehicles is 1A3b1.



AT new registered cars: https://www.data.gv.at/katalog/de/dataset/stat_pkw-neuzulassungen-nach-marken-ab-janner-2000#resources

AT used registered cars: https://www.data.gv.at/katalog/dataset/6fc1c927-bf21-3d72-9333-a9055493ab3c#additional-info

## API

API : TODO Gregoire

# Infrastructure

TODO

 ## Setup: 
 
Run the notebook locally

#### Github Repo:
https://github.com/huhubi/BDENG

### Versions used

- JupyterLab
- Python 3 as kernel
- python-Kafka
- Spark


## Expected Output :

We want to see if there is a difference between the data count from the csv files and the data from the webpage

# Documentation of procedure

# INSERT NOTEBOOK

In [ ]:
# TODO

# Conclusion